In [3]:
lines = sc.textFile("hdfs:///user/.../iris.data")

In [4]:
lines.take(10)

['5.1,3.5,1.4,0.2,Iris-setosa',
 '4.9,3.0,1.4,0.2,Iris-setosa',
 '4.7,3.2,1.3,0.2,Iris-setosa',
 '4.6,3.1,1.5,0.2,Iris-setosa',
 '5.0,3.6,1.4,0.2,Iris-setosa',
 '5.4,3.9,1.7,0.4,Iris-setosa',
 '4.6,3.4,1.4,0.3,Iris-setosa',
 '5.0,3.4,1.5,0.2,Iris-setosa',
 '4.4,2.9,1.4,0.2,Iris-setosa',
 '4.9,3.1,1.5,0.1,Iris-setosa']

In [6]:
import numpy as np

def parseVector(line):
    return np.array([float(x) for x in line.split(',')[:-1]])

In [7]:
lines_filtered = lines.filter(lambda line: len(line.split(",")) == 5)
data = lines_filtered.map(parseVector)

In [8]:
data.take(10)

[array([5.1, 3.5, 1.4, 0.2]),
 array([4.9, 3. , 1.4, 0.2]),
 array([4.7, 3.2, 1.3, 0.2]),
 array([4.6, 3.1, 1.5, 0.2]),
 array([5. , 3.6, 1.4, 0.2]),
 array([5.4, 3.9, 1.7, 0.4]),
 array([4.6, 3.4, 1.4, 0.3]),
 array([5. , 3.4, 1.5, 0.2]),
 array([4.4, 2.9, 1.4, 0.2]),
 array([4.9, 3.1, 1.5, 0.1])]

In [9]:
def closestPoint(p):
    
    centers = centers_bc.value
    
    bestIndex = 0
    closest = float("+inf")
    
    for i in range(len(centers)):
        
        tempDist = np.sum((p - centers[i]) ** 2)
        
        if tempDist < closest:
            closest = tempDist
            bestIndex = i

    return bestIndex

In [10]:
data.cache()

PythonRDD[6] at RDD at PythonRDD.scala:53

In [11]:
K = 3
max_iter = 10

centers = data.takeSample(False, K, 1)

distances = None
iteration = 0

while iteration < max_iter:
    
    centers_bc = sc.broadcast(centers) 

    closest = data.map(lambda p: (closestPoint(p), (p, 1)))
    
    stats = closest.reduceByKey(lambda p1_c1, p2_c2: (p1_c1[0] + p2_c2[0], p1_c1[1] + p2_c2[1]))
    
    new_centers_pairs = stats.map(lambda clidx_stats: (clidx_stats[0], clidx_stats[1][0] / clidx_stats[1][1]))
    new_centers_pairs = new_centers_pairs.collect()

    for (iK, p) in new_centers_pairs:
        centers[iK] = p
        
    print("Cluster centers for iteration {}:".format(iteration))
    for k in range(K):
        print("\t mean {}: {}".format(k, centers[k]))
    
    iteration += 1

Cluster centers for iteration 0:
	 mean 0: [5.99104478 2.79253731 4.56865672 1.51641791]
	 mean 1: [6.99333333 3.09666667 5.83       2.09666667]
	 mean 2: [5.00566038 3.36037736 1.56226415 0.28867925]
Cluster centers for iteration 1:
	 mean 0: [5.9453125 2.759375  4.4515625 1.44375  ]
	 mean 1: [6.87428571 3.08857143 5.79142857 2.11714286]
	 mean 2: [5.00784314 3.4        1.49411765 0.26078431]
Cluster centers for iteration 2:
	 mean 0: [5.9047619  2.74603175 4.41269841 1.43333333]
	 mean 1: [6.87027027 3.08648649 5.74594595 2.08918919]
	 mean 2: [5.006 3.418 1.464 0.244]
Cluster centers for iteration 3:
	 mean 0: [5.9016129  2.7483871  4.39354839 1.43387097]
	 mean 1: [6.85       3.07368421 5.74210526 2.07105263]
	 mean 2: [5.006 3.418 1.464 0.244]
Cluster centers for iteration 4:
	 mean 0: [5.9016129  2.7483871  4.39354839 1.43387097]
	 mean 1: [6.85       3.07368421 5.74210526 2.07105263]
	 mean 2: [5.006 3.418 1.464 0.244]
Cluster centers for iteration 5:
	 mean 0: [5.9016129  2.74